# Vitessce Widget Tutorial

## Import dependencies


https://github.com/vitessce/vitessce/blob/main/examples/configs/src/view-configs/spatial-beta/kpmp.js

In [ ]:
from os.path import join

In [ ]:
base_dir = join("..", "..", "..", "kpmp-f2f-march-2023", "S-1905-017737")

In [ ]:
sdata_path = join(base_dir, "sdata.zarr")

In [ ]:
!ls {sdata_path}/images

In [ ]:
!ls {sdata_path}/labels

In [ ]:
!ls {sdata_path}/tables

In [ ]:
!ls {sdata_path}/tables/table_cortical_interstitia/obs

In [ ]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    AnnDataWrapper,
    ImageOmeTiffWrapper,
    ImageOmeZarrWrapper,
    ObsSegmentationsOmeZarrWrapper,
    get_initial_coordination_scope_prefix,
    hconcat,
    vconcat,
)

In [ ]:
# Create a VitessceConfig instance.
vc = VitessceConfig(schema_version="1.0.17", name="SpatialData")

# Add a new dataset to the Vitessce configuration,
# then add the wrapper class instance to this dataset.
dataset = vc.add_dataset(name='KPMP').add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        image_path="images/image",
        coordinate_system="global",
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        table_path="tables/table_tubules",
        labels_path="labels/labels_tubules",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_tubules",
          "obsType": "Tubule"   
        }
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        labels_path="labels/labels_arteries_arterioles",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_arteries_arterioles",
          "obsType": "Artery"   
        }
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        table_path="tables/table_cortical_interstitia",
        labels_path="labels/labels_cortical_interstitia",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_cortical_interstitia",
          "obsType": "Cortical Interstitium"   
        }
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        table_path="tables/table_globally_sclerotic_glomeruli",
        labels_path="labels/labels_globally_sclerotic_glomeruli",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_globally_sclerotic_glomeruli",
          "obsType": "Globally Sclerotic Glomerulus"   
        }
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        table_path="tables/table_non_globally_sclerotic_glomeruli",
        labels_path="labels/labels_non_globally_sclerotic_glomeruli",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_non_globally_sclerotic_glomeruli",
          "obsType": "Non-Globally Sclerotic Glomerulus"   
        }
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        table_path="tables/table_interstitialfibrosis_and_tubular_atrophy",
        labels_path="labels/labels_interstitialfibrosis_and_tubular_atrophy",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_interstitialfibrosis_and_tubular_atrophy",
          "obsType": "Interstitial Fibrosis and Tubular Atrophy"   
        }
    )
).add_object(
    SpatialDataWrapper(
        sdata_path=sdata_path,
        table_path="tables/table_peritubular_capillaries",
        labels_path="labels/labels_peritubular_capillaries",
        #obs_feature_matrix_path="tables/table/X",
        #obs_set_paths=["tables/table/obs/annotation"],
        #obs_set_names=["Annotation"],
        #region="tubules",
        coordinate_system="global",
        coordination_values={
          "fileUid": "labels_peritubular_capillaries",
          "obsType": "Peritubular Capillaries"   
        }
    )
)

# Add views (visualizations) to the configuration.
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view("featureList", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
obs_sets = vc.add_view("obsSets", dataset=dataset)
heatmap = vc.add_view("heatmap", dataset=dataset)

[obs_color_encoding_scope] = vc.add_coordination("obsColorEncoding")
obs_color_encoding_scope.set_value("cellSetSelection")

vc.link_views_by_dict([spatial, layer_controller], {
    "imageLayer": CL([{
        "photometricInterpretation": "RGB",
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))

vc.link_views_by_dict([spatial, layer_controller], {
    "segmentationLayer": CL([
        {
            "fileUid": "labels_tubules",
            "segmentationChannel": CL([{
                "obsType": "Tubule",
                "spatialChannelVisible": True,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        },
        {
            "fileUid": "labels_arteries_arterioles",
            "segmentationChannel": CL([{
                "obsType": "Artery",
                "spatialChannelVisible": True,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        },
        {
            "fileUid": "labels_cortical_interstitia",
            "segmentationChannel": CL([{
                "obsType": "Cortical Interstitium",
                "spatialChannelVisible": False,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        },
        {
            "fileUid": "labels_globally_sclerotic_glomeruli",
            "segmentationChannel": CL([{
                "obsType": "Globally Sclerotic Glomerulus",
                "spatialChannelVisible": False,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        },
        {
            "fileUid": "labels_non_globally_sclerotic_glomeruli",
            "segmentationChannel": CL([{
                "obsType": "Non-Globally Sclerotic Glomerulus",
                "spatialChannelVisible": False,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        },
        {
            "fileUid": "labels_interstitialfibrosis_and_tubular_atrophy",
            "segmentationChannel": CL([{
                "obsType": "Interstitial Fibrosis and Tubular Atrophy",
                "spatialChannelVisible": False,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        },
        {
            "fileUid": "labels_peritubular_capillaries",
            "segmentationChannel": CL([{
                "obsType": "Peritubular Capillaries",
                "spatialChannelVisible": False,
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 0.5,
                "obsColorEncoding": "spatialChannelColor",
            }]),
        }
    ]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSegmentations"))

vc.link_views([spatial, layer_controller, feature_list, obs_sets, heatmap], [], [])

# Layout the views in a grid arrangement.
vc.layout((spatial / heatmap) | (layer_controller / (feature_list | obs_sets)));

In [ ]:
vw = vc.widget()
vw